In [1]:
# File names
name_pre = 'b'
name_end = '_stats_toothpick_v1.1.fits' 
name_list = ["{}{:02}{}".format(name_pre, i, name_end) for i in range(2,24)]
filepath = '/home/ilion/0/qd174/Git/PHAT_BEAST/'

In [62]:
from astropy.io import fits
data = np.array([fits.open(filepath + filename, memmap=True)[1] for filename in name_list])
column_names = data[0].columns.names

In [65]:
column_names.sort()
column_names

['Av_Best',
 'Av_Exp',
 'Av_p16',
 'Av_p50',
 'Av_p84',
 'DEC',
 'HST_ACS_WFC_F475W',
 'HST_ACS_WFC_F814W',
 'HST_WFC3_F110W',
 'HST_WFC3_F160W',
 'HST_WFC3_F275W',
 'HST_WFC3_F336W',
 'M_act_Best',
 'M_act_Exp',
 'M_act_p16',
 'M_act_p50',
 'M_act_p84',
 'M_ini_Best',
 'M_ini_Exp',
 'M_ini_p16',
 'M_ini_p50',
 'M_ini_p84',
 'Name',
 'Pmax',
 'Pmax_indx',
 'RA',
 'Rv_A_Best',
 'Rv_A_Exp',
 'Rv_A_p16',
 'Rv_A_p50',
 'Rv_A_p84',
 'Rv_Best',
 'Rv_Exp',
 'Rv_p16',
 'Rv_p50',
 'Rv_p84',
 'Z_Best',
 'Z_Exp',
 'Z_p16',
 'Z_p50',
 'Z_p84',
 'chi2min',
 'chi2min_indx',
 'f_A_Best',
 'f_A_Exp',
 'f_A_p16',
 'f_A_p50',
 'f_A_p84',
 'field',
 'inside_brick',
 'inside_chipgap',
 'logA_Best',
 'logA_Exp',
 'logA_p16',
 'logA_p50',
 'logA_p84',
 'logF_QION_nd_Best',
 'logF_QION_nd_Exp',
 'logF_QION_nd_p16',
 'logF_QION_nd_p50',
 'logF_QION_nd_p84',
 'logF_QION_wd_Best',
 'logF_QION_wd_Exp',
 'logF_QION_wd_p16',
 'logF_QION_wd_p50',
 'logF_QION_wd_p84',
 'logF_UV_6_13e_nd_Best',
 'logF_UV_6_13e_nd_Exp

In [11]:
import sys
sys.getsizeof(data)

264

In [2]:
import numpy as np

In [25]:
brick_idx = [2,5]
np.concatenate(
    [
        data[brick_i].data['DEC'][:20]
        for brick_i in brick_idx
    ]
)

array([41.30651666, 41.35671469, 41.24782826, 41.33286783, 41.26621411,
       41.18271918, 41.23190887, 41.34401198, 41.3037616 , 41.23138694,
       41.18189154, 41.24738839, 41.33429234, 41.27289648, 41.33423943,
       41.2318987 , 41.30650537, 41.26886428, 41.25211132, 41.20722426,
       41.53384452, 41.59793725, 41.4476699 , 41.50750076, 41.45197925,
       41.58960365, 41.60635193, 41.49505176, 41.61183941, 41.60785268,
       41.59809449, 41.53350575, 41.60073967, 41.48533642, 41.48669443,
       41.45180462, 41.56190571, 41.56189467, 41.55068773, 41.50616151])

In [20]:
brick_idx

[2, 5]

In [29]:
fits.open(filepath + name_list[0], memmap=True)[1].header['NAXIS2']

2296520

In [59]:
import numba as nb
#@nb.jit(nb.f8[:](nb.f8, nb.f8), nopython=True)
def getSampleIndices(sample_size, total_size):
    """ 
    Returns data points for the subsample in the range
    Timed Tests:   data_count = 100,000,000;  sample_size = 100,000;
        926 us  np.random.randint(low=0, high=data_count+1, size=sample_size) 
        935 us  np.random.choice(data_count, size=sample_size, replace=True, p=None)
         85 ms  random.sample(range(data_count), sample_size)
          6 s   np.random.choice(data_count, size=sample_size, replace=False, p=None)
    """
    # note: allows multiple identical values, hence n<=num_points, but orders mag faster
    select_points = np.random.randint(
                        low=0, high=total_size+1, 
                        size=sample_size
                    )
    unique_pts = np.unique(select_points)
    unique_pts_count = len(unique_pts)
    select_points[:unique_pts_count] = unique_pts
    while len(select_points) != sample_size:
        # TODO: In theory go for more and then only add as needed 
        select_points[unique_pts_count:] = np.random.randint(low=0, high=total_size+1, 
                                                  size=sample_size-unique_pts_count )
        unique_pts = np.unique(select_points)
        unique_pts_count = len(unique_pts)

    return select_points

@nb.jit(nb.f8(nb.f8,nb.f8), nopython=True)
def getExpected(sample_size, range_size):
    return (1-(1-1/range_size)**sample_size)*sample_size

@nb.jit(nb.f8(nb.f8,nb.f8), nopython=True)
def getNeeded(sample_size, range_size):
    return np.log(1-sample_size/range_size)/np.log(1-1/range_size)

def getSampleIndices(sample_size, total_size):
    """ 
    Returns data points for the subsample in the range
    Timed Tests:   data_count = 100,000,000;  sample_size = 100,000;
        926 us  np.random.randint(low=0, high=data_count+1, size=sample_size) 
        935 us  np.random.choice(data_count, size=sample_size, replace=True, p=None)
         85 ms  random.sample(range(data_count), sample_size)
          6 s   np.random.choice(data_count, size=sample_size, replace=False, p=None)
    """
    # Expected number of unique values
    exp_sample_size = round(getExpected(sample_size, total_size))
    if not exp_sample_size == sample_size:
        # Adjust sample_size
        adj_sample_size = round(getNeeded(sample_size, total_size)*1.1)
    else:
        adj_sample_size = sample_size

    select_points = np.random.randint(
                        low=0, high=total_size+1, 
                        size=adj_sample_size
                    )
    unique_pts = np.unique(select_points)
    unique_pts_count = len(unique_pts)
    print(unique_pts_count)
#    while len(select_points) != sample_size:
#        select_points[:unique_pts_count] = unique_pts
#        select_points[unique_pts_count:] = np.random.randint(low=0, high=total_size+1, 
#                                                  size=sample_size-unique_pts_count )
#        unique_pts = np.unique(select_points)
#        unique_pts_count = len(unique_pts)

    return select_points[:sample_size-1]


In [52]:

# Probability of duplicates
n = 10
k = 7
exp1 = (1-(1-1/n)**k)*n  
k = round(np.log(1-k/n)/np.log(1-1/n)+1)
exp2 = (1-(1-1/n)**k)*n
exp1, exp2

(5.217030999999999, 7.175704635189999)

In [4]:
data_count = 100000000
sample_size = 100000

In [3]:
import random

In [60]:
getSampleIndices(data_count, sample_size)

100001


array([18782, 86579, 47038, ..., 40011, 39610, 64993])

In [55]:
%%timeit
selection = getSampleIndices(data_count, sample_size)

unique_pts_count
unique_pts_count
unique_pts_count
unique_pts_count
unique_pts_count
unique_pts_count
unique_pts_count
unique_pts_count
8.49 s ± 454 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
selection = np.random.randint(low=0, high=data_count+1, size=sample_size)
len(selection)

numpy.ndarray

In [6]:
%%timeit
selection = random.sample(range(data_count), sample_size)

100 ms ± 7.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%%timeit
selection = np.random.choice(data_count, size=sample_size, replace=False, p=None)

5.78 s ± 391 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%timeit
selection = np.random.choice(data_count, size=sample_size, replace=True, p=None)

975 µs ± 15.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [39]:
%%timeit  # not unique
selection = np.random.randint(low=0, high=data_count+1, size=sample_size)

979 µs ± 19.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [35]:
%%timeit  # not unique
selection = np.unique(np.random.randint(low=0, high=data_count+1, size=sample_size))

6.98 ms ± 60.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
selection = random.sample(range(data_count), sample_size)
len(np.unique(selection))

10000

In [32]:
selection = np.random.choice(data_count, size=sample_size, replace=False, p=None)
len(np.unique(selection))

10000

In [48]:
selection = np.random.randint(low=0, high=data_count, size=sample_size)
len(np.unique(selection))

10000

In [35]:
selection = np.random.rand(sample_size, data_count])
len(np.unique(selection))

TypeError: 'list' object cannot be interpreted as an integer

In [9]:
["a", "b"] in ["a", "b", "c"]

False